In [ ]:
# -*- coding: utf-8 -*- In TensorFlow
"""Assignment2_tensor.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1UjqhL-oDzOuvfZ7wKzKS7Qkx9WFRSWWm
"""

!pip install talos
import talos
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend
from tensorflow.keras.activations import softmax, sigmoid, relu
from tensorflow.keras.optimizers import Adam
import pickle
from talos import Scan, Evaluate, Deploy

with open('train', 'rb') as file:
    train_dict = pickle.load(file, encoding='bytes')

with open('test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

X_train = train_dict[b'data']
y_train = train_dict[b'coarse_labels']

X_test = test_dict[b'data']
y_test = test_dict[b'coarse_labels']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

p = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'activation': ['softmax', 'sigmoid'],
    'loss': ['mse', 'categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [1000, 2000]
}

def create_model(X_train,y_train,X_test,y_test,params):
  model = Sequential()
  model.add(Dense(units=params['units'], input_shape = X_train.shape[1:]))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=100, activation=params['activation']))

  model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])

  out = model.fit(x=X_train, y=y_train, batch_size=params['batch_size'], epochs=200)

  return out,model

t = talos.Scan(x=X_train, y=y_train, params=p, model=create_model, x_val=X_test, y_val=y_test, experiment_name="talos_output")
#e = talos.Evaluate(t)
#evaluation = e.evaluate(X_test, y_test, metric = 'accuracy')
#model = Deploy(t, model_name = 'CIFAR100', metric = 'accuracy')

#t = talos.Scan(x=X_train, y=y_train, params=p, model=create_model, x_val=X_test, y_val=y_test, experiment_name="talos_output")
e = talos.Evaluate(t)
print("Evaluation")
#result = model.evaluate(X_test, y_test)
evaluation = e.evaluate(X_test, y_test, metric = 'accuracy', task = 'multi_label')
print(evaluation[0])
print(evaluation[1])
print(evaluation[2])
print(evaluation[3])
print(evaluation[4])
model = Deploy(t, model_name = 'CIFAR100', metric = 'accuracy')





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


ModuleNotFoundError: ignored

In [ ]:
# -*- coding: utf-8 -*-   In PyTORCH
"""Assingment2_Pytorch.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1uZ0K0E5PVM-RQ_od99KGpBcmpJlw0RXv
"""

!pip install pytorch_lightning
!pip install talos
import talos
import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision.datasets import MNIST
from torchvision import transforms
import pickle
from pytorch_lightning import Trainer, seed_everything, LightningModule
from pytorch_lightning.callbacks import EarlyStopping
from talos import Scan, Evaluate, Deploy

with open('train', 'rb') as file:
    train_dict = pickle.load(file, encoding='bytes')

with open('test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

X_train = train_dict[b'data']
y_train = train_dict[b'coarse_labels']

X_test = test_dict[b'data']
y_test = test_dict[b'coarse_labels']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = torch.tensor(y_train, dtype=torch.float32)
y_train = y_train.view(-1, 1) # reshape to column tensor
y_train = y_train.expand(-1, 100) # replicate the column tensor to a matrix of size [batch_size, 100]

y_test = torch.tensor(y_test, dtype=torch.float32)
y_test = y_test.view(-1, 1)
y_test = y_test.expand(-1, 100)

#y_train = torch.tensor(y_train, dtype=torch.int64)
#y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(torch.tensor(X_train), y_train)
test_dataset = TensorDataset(torch.tensor(X_test), y_test)

p = {
    'hidden_dim': [120, 240],
    'hidden_activations': [F.relu, F.sigmoid],
    'output_activation': [F.softmax, F.sigmoid],
    'loss': [F.mse_loss, torch.nn.CrossEntropyLoss()],
    'optimizer': [torch.optim.Adam, torch.optim.Adagrad],
    'batch_size': [1000, 2000]
}

class Net(nn.Module):
    def __init__(self, hidden_dim, hidden_activation, output_activation):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3072, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, hidden_dim)
        self.fc5 = nn.Linear(hidden_dim, hidden_dim)
        self.fc6 = nn.Linear(hidden_dim, 100)
        self.hidden_activation = hidden_activation
        self.output_activation = output_activation

    def forward(self, x):
        x = x.view(-1, 3072)  # flatten the input tensor
        x = self.hidden_activation(self.fc1(x))
        x = self.hidden_activation(self.fc2(x))
        x = self.hidden_activation(self.fc3(x))
        x = self.hidden_activation(self.fc4(x))
        x = self.hidden_activation(self.fc5(x))
        x = self.output_activation(self.fc6(x))
        return x

#def create_model(X_train, y_train, X_test, y_test, params):
 #   net = Net(params['hidden_dim'], params['hidden_activations'], params['output_activation'])
  #  criterion = params['loss']
   # optimizer = params['optimizer'](net.parameters())
  #  batch_size = params['batch_size']
  #  train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
 #   train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#    for epoch in range(200):
#        running_loss = 0.0
  #      for i, data in enumerate(train_loader, 0):
 #           inputs, labels = data
  #          optimizer.zero_grad()
  #          outputs = net(inputs)
  #          loss = criterion(outputs, labels)
 #           loss.backward()
 #           optimizer.step()
 #           running_loss += loss.item()
#            print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))
#    return net

    # Use GPU if available
#    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#    net.to(device)
def create_model(X_train, y_train, X_test, y_test, params):
    net = Net(params['hidden_dim'], params['hidden_activations'], params['output_activation'])
    criterion = params['loss']
    optimizer = params['optimizer'](net.parameters())
    batch_size = params['batch_size']
    train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(y_train))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



    for epoch in range(200):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            #print('[Epoch %d] loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

    return net

t = talos.Scan(x=X_train, y=y_train, params=p, model=create_model, x_val=X_test, y_val=y_test, experiment_name="talos_output")
e = talos.Evaluate(t)
evaluation = e.evaluate(X_test, y_test, metric = 'accuracy')
model = Deploy(t, model_name = 'CIFAR100', metric = 'accuracy')